#### Distribución de frecuencia 
Es el agrupamiento de variables que se utiliza para describir la frecuencia de cada una en un conjunto. Pueden calcularse como valores decimales entre 0 y 1, donde cada valor representa su proporción asociada en las observaciones y la probabilidad de que un objeto aleatorio pertenezca a esa categoría.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score


df = pd.read_csv("data/student_performance.csv")

# ignoramos el identificador (StudentID) y la variable objetivo (GradeClass)

for c in df.columns[1:-1]:
    unique_values = df[c].unique()
    if len(unique_values) == 2:
        print(f"Valores únicos en '{c}': {df[c].unique()} (Binario)\n---------------------------------------------------------------------------------------------------------")
    else:
        print(f"Valores únicos en '{c}': {df[c].unique()}\n---------------------------------------------------------------------------------------------------------")

#### Dsitribución de frecuencia en categóricas binarias

In [ ]:
def percentage_binary(df, variable, category_1, category_2):
    variable_mean = df.groupby("GradeClass")[variable].mean().reset_index(name=f"{variable}_mean")
    variable_mean[f"{variable}_mean"] = variable_mean[f"{variable}_mean"] * 100 
    for v in variable_mean.values:
        percentage = 100 - v[1]
        if percentage >= 50:
            percentage = percentage.astype(str)
            print(f"En el grado clase {v[0]} el {percentage[:4]}% de los alumnos {category_1}")
        else:
            percentage = v[1].astype(str)
            print(f"En el grado clase {v[0]} el {percentage[:4]}% de los alumnos {category_2}")
    print("-------------------------------------------------------------------------------------------------------------")
            
gender_percentage = percentage_binary(df,"Gender","son Hombres","son Mujeres")
tutoring_percentage = percentage_binary(df,"Tutoring","no tiene tutoría","tiene tutoría")
extracurricular_percentage = percentage_binary(df,"Extracurricular","no participa en actividades extracurriculares","participa actividades extracurriculares")
sports_percentage = percentage_binary(df,"Sports","no participa en deportes","participa en deportes")
music_percentage = percentage_binary(df,"Music","no participa en actividades musicales","participa en actividades musicales")
volunteering_percentage = percentage_binary(df,"Volunteering","no participa en voluntariado","participa en voluntariado")

#### Distribución de frecuencia en variables categóricas (nominales y ordinales)

In [ ]:
def percentage(df, variable, categories):
    unique_values = df["GradeClass"].unique()
    for g in np.sort(unique_values):
        grade_class = df[df["GradeClass"] == g]
        variable_count = grade_class[variable].value_counts().reset_index()
        variable_count[variable] = categories
        print(f"{variable} en grado clase {g}")
        for p in variable_count.values:
            percentage = (p[1] / variable_count["count"].sum()) * 100
            percentage = percentage.astype(str)
            print(f"{p[0]}: {percentage[:4]}%")
        print("----------------------------------------")
        
ethnicity_percentage = percentage(df,"Ethnicity",["caucásicos","áfricanos","asiáticos","otros"])
parental_education_percentage = percentage(df,"ParentalEducation",["ninguno","escuela secundaria","un poco de universidad","licenciatura","superior"]) 
parental_support_percentage = percentage(df,"ParentalSupport",["ninguno","bajo","moderado","alto","muy alto"])

#### Bagging 
es una de las principales técnicas computacionales de estadística inferencial cuya base es utilizar la sabiduría de las masas, es decir, reforzar el aprendizaje mediante el criterio de distintos metaestimadores que se encargarán de sacar sus propias conclusiones de respectivos datos seleccionados para luego, dependiendo del problema en cuestión, llegar a una respuesta final a través de la media o moda del conjunto de resultados.
_________________________________________________________________________________________________________

#### Naive Bayes 
Algoritmo de clasificación probabilístico basado en el teorema de Bayes y en la suposición de que las características son condicionalmente independientes entre si dado la clase a la que pertenecen, lo que significa que la probabilidad de una característica no depende de las otras, pero si aportan información sobre la variable objetivo. Además, supone que las variables independientes siguen una distribución específica, permitiendo calcular la probabilidad final de que la variable objetivo este asociada a esa clase. 

Suposiciones del algoritmo:
- Independencia condicional entre las características
- Distribuciones conocidas para modelar en las características(normal, binomial, etc.)
- No hay correlación entre características
- Clases mutuamente excluyentes
- No hay ruido en los datos

In [ ]:
# Discretización de la variable continua 'StudyTimeWeekly'
Q1 = np.percentile(df["StudyTimeWeekly"], 25)
Q2 = np.percentile(df["StudyTimeWeekly"], 50)
Q3 = np.percentile(df["StudyTimeWeekly"], 75)
Q4 = np.percentile(df["StudyTimeWeekly"], 100)

df["StudyTimeWeekly"] = np.where(df["StudyTimeWeekly"] <= Q1, Q1, 
                        np.where((df["StudyTimeWeekly"] > Q1) & (df["StudyTimeWeekly"] <= Q2), Q2, 
                        np.where((df["StudyTimeWeekly"] > Q2) & (df["StudyTimeWeekly"] <= Q3), Q3, 
                        df["StudyTimeWeekly"])))

df["StudyTimeWeekly"] = round(df["StudyTimeWeekly"])

#------------------------------------------------------------------------------------------------------
naive_bayes = GaussianNB()

bagging_model = BaggingClassifier(estimator=naive_bayes, # clasificador base
                        n_estimators=100, # cantidad de estimadores
                        max_samples=0.3,# número de muestras requeridas para cada modelo
                        oob_score=True) # se usan como datos de prueba aquellos que no fueron seleccionados

var_x = df[df.columns[1:-1]]
var_y = df["GradeClass"]

bagging_model.fit(var_x, var_y) 
print(f"Accuracy mean: {bagging_model.oob_score_}")

In [ ]:
object = np.array([])

def create_object(array, df):
    for var in df.columns[1:-1]:
        random_value = np.random.choice(df[var].unique())
        array = np.append(array, random_value)
    
    return array

object = create_object(object, df)

estimators_predicts = [estimator.predict(object.reshape(1,13)) for estimator in bagging_model.estimators_]

estimators_predicts_array = np.array(estimators_predicts).reshape(-1)

count_estimators_predicts = np.unique(estimators_predicts_array, return_counts=True)

plt.figure(figsize=(14,5))
plt.bar(count_estimators_predicts[0], count_estimators_predicts[1])
plt.title("Conteo de predicciones de los estimadores(100)")
plt.xlabel("Respuestas asociadas")
plt.ylabel("Conteo de clases")
plt.show()

#### Dashboard que refleja la distribución de categorías nominales y ordinales


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e6_body",children=[
    html.H1("Categorías de estudiantes",id="title",className="e6_title"),
    html.Div(id="div",className="e6_div",children=[
        html.Div(id="div_dropdown",className="e6_div_dropdown",children=[
            dcc.Dropdown(id="dropdown_1",className="e6_dropdown",
                        options = [
                            {"label":"Grado clase 0","value":0},
                            {"label":"Grado clase 1","value":1},
                            {"label":"Grado clase 2","value":2},
                            {"label":"Grado clase 3","value":3},
                            {"label":"Grado clase 4","value":4},
                        ],
                        value=0,
                        multi=False,
                        clearable=False),
            dcc.Dropdown(id="dropdown_2",className="e6_dropdown",
                        options = [
                            {"label":"Género","value":"Gender"},
                            {"label":"Tutoría","value":"Tutoring"},
                            {"label":"Actividades extracurriculares","value":"Extracurricular"},
                            {"label":"Deportes","value":"Sports"},
                            {"label":"Música","value":"Music"},
                            {"label":"Voluntariado","value":"Volunteering"},
                            {"label":"Etnia","value":"Ethnicity"},
                            {"label":"Educación paternal","value":"ParentalEducation"},
                            {"label":"Apoyo paternal","value":"ParentalSupport"},
                        ],
                        value="Gender",
                        multi=False,
                        clearable=False)
        ])
    ]),
    dcc.Graph(id="graph",className="e6_graph",figure={})
])

@app.callback(
    Output(component_id="graph",component_property="figure"),
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

def update_graph(slct_var_obj,slct_var_car):
    
    grade_class = df[df["GradeClass"] == slct_var_obj]
    variable_count = grade_class[slct_var_car].value_counts().reset_index()
    variable_count.sort_values(slct_var_car, inplace=True)
        
    if slct_var_car == "Gender":
        variable_count[slct_var_car] = ["Hombre","Mujer"]
    elif slct_var_car == "Tutoring":
        variable_count[slct_var_car] = ["Sin tutoría","Tutoría"]
    elif slct_var_car == "Extracurricular":
        variable_count[slct_var_car] = ["No participa en actividades extracurriculares","Participa en actividades extracurriculares"]
    elif slct_var_car == "Sports":
        variable_count[slct_var_car] = ["No participa en deportes","Participa en deportes"]
    elif slct_var_car == "Music":
        variable_count[slct_var_car] = ["No participa en actividades musicales","Participa en actividades musicales"]
    elif slct_var_car == "Volunteering":
        variable_count[slct_var_car] = ["No participa en voluntariado","Participa en voluntariado"]
    elif slct_var_car == "Ethnicity":
        variable_count[slct_var_car] = ["caucásicos","áfricanos","asiáticos","otros"]
    elif slct_var_car == "ParentalEducation":
        variable_count[slct_var_car] = ["ninguno","escuela secundaria","un poco de universidad","licenciatura","superior"]
    elif slct_var_car == "ParentalSupport":
        variable_count[slct_var_car] = ["ninguno","bajo","moderado","alto","muy alto"]
    
    piechart = px.pie(variable_count, values="count", names=slct_var_car, title=f"Distribución de {slct_var_car} en Grado Clase {slct_var_obj}")

    return piechart

if __name__ == "__main__":
    app.run_server(debug=False)

#### Automatización de Toma de Decisiones
Siendo una de los principales desarrollos de DS y cuyo objetivo es ayudar a ahorrar tiempo y recursos, reducir errores humanos y mejorar la eficiencia. En este proyecto no solo se elaboraron informes que explican como están distribuidas las propiedades de los alumnos de diferentes clases de grado, sino que también se ha realizado automatización de tareas mediante un modelo predictivo que, basándose en los distinto datos conocidos, es capaz de asignar a un nuevo alumno su clase de grado correspondiente, en un presunto caso ahorrándole este trabajo a la organización encargada de gestionar esta cuestión.